# Schrödinger equation

## Estática

### Ejemplo del artículo

http://pubs.acs.org/doi/pdf/10.1021/acs.jchemed.7b00003

In [ ]:
%matplotlib inline

import scipy as sci
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt

plt.rc('figure', figsize=(4,3))

import cv2
from umucv.htrans import null1

In [ ]:
def mkLaplacian(n,dx,periodic=True):
    Lap = np.diag(-2*np.ones(n)) + np.diag(np.ones(n-1),1) + np.diag(np.ones(n-1),-1)
    if periodic:
        Lap[0,-1]=1
        Lap[-1,0]=1
    return Lap/dx**2

In [ ]:
m = 1
hbar = 1

In [ ]:
n = 500
x = np.linspace(-2,2,n)
dx = x[1]-x[0]
Lap = mkLaplacian(n,dx)

In [ ]:
V =  0*x - 100*(x>-0.5)  + 100*(x>0.5)

H = np.diag(V) - (hbar**2/2/m)*Lap

In [ ]:
plt.plot(x,V); plt.xlabel('x'); plt.ylabel('V(x)');

In [ ]:
plt.imshow(H[:20,:20],'gray');

In [ ]:
E,f = la.eigh(H)

In [ ]:
E[:10]

In [ ]:
plt.figure(figsize=(6,6))
for k in range(5):
    plt.plot(x,E[k]+400*(f[:,k]**2))
plt.plot(x,V,color='gray');

### Ejemplo del libro

In [ ]:
hbar = 1
m = 1
V0 = 32

V =  V0*(x>0.5) +V0*(x<-0.5)

H = np.diag(V) - (hbar**2/2/m)*Lap

plt.plot(x,V);

In [ ]:
E,f = la.eigh(H)

In [ ]:
E[:10]

In [ ]:
E[:10]/V0

Resultado analítico:

    0.0977   0.383    0.810

In [ ]:
plt.figure(figsize=(6,6))
for k in range(0,4):
    plt.plot(x,E[k]+400*(f[:,k]**2))
plt.plot(x,V,color='gray');

In [ ]:
Hg = H - 34.4084826*np.eye(len(H))

In [ ]:
Hg = H - 60*np.eye(len(H))

In [ ]:
u,s,v = la.svd(Hg)

In [ ]:
s[:10], s[-10:]

In [ ]:
plt.plot(x,null1(Hg),x,f[:,4]+0.001);

## Dinámica

### Pruebas iniciales

Excelente animación: https://phet.colorado.edu/en/simulation/quantum-tunneling

Y esto: http://sites.tufts.edu/softmattertheory/2012/12/21/visualizing-quantum-mechanics-with-mathematica/

In [ ]:
def wave(k,x):
    return np.exp(1j*k*x)

In [ ]:
def Nor(f):
    k = dx*sum(abs(f)**2)
    return f/np.sqrt(k)

In [ ]:
def expmi(m):
    l,v = la.eigh(m)
    c = v @ np.diag(np.cos(l)) @ v.conj().T
    s = v @ np.diag(np.sin(l)) @ v.conj().T
    return c + 1j*s

In [ ]:
km = 10
dk = 3

ks = np.linspace(km-5*dk,km+5*dk,30)
g = np.exp(-0.5*((ks-km)/dk)**2) 
plt.plot(ks,g);

In [ ]:
n = 500
x = np.linspace(-2,2,n)
dx = x[1]-x[0]
Lap = mkLaplacian(n,dx)

In [ ]:
packet = sum([a*wave(k,x) for a,k in zip(g,ks)])
packet = Nor(packet)
dx*sum(abs(packet)**2)

In [ ]:
plt.plot(x,np.real(packet),x,np.imag(packet));

In [ ]:
plt.plot(x,abs(packet)**2);

In [ ]:
hbar = 1
m = 1
V0 = 32

# V =  V0*(x>0.5) +V0*(x<-0.5)
V = 0*x

H = np.diag(V) - (hbar**2/2/m)*Lap

In [ ]:
pp = (H @ packet)/(1j*hbar)

plt.plot(x,np.real(pp),x,np.imag(pp));

In [ ]:
prop = expmi(-1/hbar*H*0.05)

In [ ]:
plt.plot(x,abs(packet)**2, x, abs(prop@packet)**2, x, abs(prop@prop@packet)**2, x, abs(prop@prop@prop@packet)**2);

In [ ]:
triang = x * (x>0)*(x<1) + (2-x)*(x>1)*(x<2)
triang = Nor(triang)
plt.plot(x,triang);

In [ ]:
plt.plot(x,abs(triang)**2, x, abs(prop@triang)**2, x, abs(prop@prop@triang)**2, x, abs(prop@prop@prop@triang)**2);

In [ ]:
plt.plot(x,np.real(prop@triang),x,np.imag(prop@triang));

In [ ]:
fun = wave(0.5*2*np.pi,x)
#fun = fun * np.exp(-0.5*((x-0)/0.2)**2)
plt.plot(x,np.real(fun),x,np.imag(fun),x,abs(fun)**2);
plt.grid();

In [ ]:
plt.plot(x,np.real(fun), x, np.real(prop@fun), x, np.real(prop@prop@fun));

In [ ]:
fun = wave(0.5*2*np.pi,x)
fun = fun * np.exp(-0.5*((x-0)/0.2)**2)
plt.plot(x,np.real(fun),x,np.imag(fun),x,abs(fun)**2);
plt.grid();

In [ ]:
plt.plot(x,np.real(fun), x, np.real(prop@fun), x, np.real(prop@prop@fun));

In [ ]:
plt.plot(x,abs(fun)**2, x, abs(prop@fun)**2, x, abs(prop@prop@fun)**2, x, abs(prop@prop@prop@fun)**2);

In [ ]:
fun = wave(2*2*np.pi,x)
fun = fun * np.exp(-0.5*((x-0)/0.2)**2)
plt.plot(x,np.real(fun),x,np.imag(fun),x,abs(fun)**2);
plt.grid();

In [ ]:
plt.plot(x,np.real(fun), x, np.real(prop@fun), x, np.real(prop@prop@fun));

In [ ]:
plt.plot(x,abs(fun)**2, x, abs(prop@fun)**2, x, abs(prop@prop@fun)**2, x, abs(prop@prop@prop@fun)**2);

In [ ]:
fun = wave(0*2*np.pi,x)
fun = fun * np.exp(-0.5*((x-0)/0.2)**2)
plt.plot(x,np.real(fun),x,np.imag(fun),x,abs(fun)**2);
plt.grid();

In [ ]:
plt.plot(x,abs(fun)**2, x, abs(prop@fun)**2, x, abs(prop@prop@fun)**2, x, abs(prop@prop@prop@fun)**2);

In [ ]:
plt.plot(x,np.real(fun), x, np.real(prop@fun), x, np.real(prop@prop@fun));

### Código para las animaciones

In [ ]:
plt.rc('animation', html='html5')
import sys
if 'pyodide' in sys.modules:
    %pip install ipywidgets
    %pip install ipympl

def metaAnimation(fig, fotogram, nframes, video=True, sleep=1/50):
    if video:
        def create(frames,interval):
            from matplotlib import animation
            return animation.FuncAnimation(fig, fotogram, frames=frames, interval=interval, blit=True, repeat=False)
        return create
    else:
        fig.canvas.toolbar_visible = False
        fig.canvas.header_visible = False
        fig.canvas.footer_visible = False
        fig.canvas.capture_scroll = False

        import time
        def play(n):
            for k in range(n):
                fotogram(k)
                fig.canvas.draw()
                time.sleep(sleep)

        import ipywidgets as widgets
        play_button =  widgets.Button(description="Play")
        play_button.on_click(lambda b: play(nframes))
        display(play_button)

        return play

In [ ]:
def makeAnimation(packet, prop, nframes=100, init=None, video=True):

    fig, ax = plt.subplots(figsize=(8,4))

    ax.set_xlim(( -4, 5))
    ax.set_ylim((-0.05, 2))

    if video:
        plt.close()

    line1, = ax.plot([], [], 'black',lw=2)
    line2, = ax.plot([],[],'blue',alpha=0.5)
    line3, = ax.plot([],[],'red',alpha=0.5)
    #line2, = ax.plot(x, V, 'gray')

    r = packet

    def fotogram(i):
        nonlocal r
        line2.set_data(x,np.real(r)/2+1)
        line3.set_data(x,np.imag(r)/2+1)
        line1.set_data(x,abs(r)**2)
        if i > 0: r = prop@r
        return ()

    if init is not None:
        fotogram(init)

    return metaAnimation(fig,fotogram,nframes,video)

In [ ]:
def makeAnimation2(packet, prop, nframes=100, init=None, video=True):

    fig, ax = plt.subplots(figsize=(8,4))

    ax.set_xlim(( -4, 5))
    ax.set_ylim((-0.05, 2))

    if video:
        plt.close()

    line1, = ax.plot([], [], '-')
    line2, = ax.plot(x, V, 'gray')

    r = packet

    def fotogram(i):
        nonlocal r
        line1.set_data(x,abs(r)**2)
        if i > 0: r = prop@r
        return ()

    if init is not None:
        fotogram(init)

    return metaAnimation(fig,fotogram,nframes,video)

In [ ]:
def makeAnimation3(packet, prop, mask=None, nframes=100, init=None, video=True):

    fig, ax = plt.subplots(figsize=(8,4))
    if video: plt.close()

    ax.set_xlim(( -4, 5))
    ax.set_ylim((-0.05, 2))


    line1, = ax.plot([], [], '-', lw=2)
    line2, = ax.plot(x, V, 'gray')

    if mask is not None:
        ax.plot(x,mask,color='pink')
    else:
        mask = 1

    r = packet

    def fotogram(i):
        nonlocal r
        line1.set_data(x,abs(r)**2)
        if i > 0:
            r = prop@r
            r = r*mask
        return ()

    if init is not None:
        fotogram(init)

    return metaAnimation(fig,fotogram,nframes,video)

In [ ]:
from IPython.display import HTML

def metadisplay(name, maker, args, nframes = 100, frames=100, interval=1000/25):
    if ANIM:
        %matplotlib widget
        maker(*args, nframes=nframes, init=0, video=False)

    tag = f"<video src='{name}.mp4' controls>video</video>"
    if MKVIDEO:
        %matplotlib inline
        create = maker(*args)
        anim = create(frames=frames, interval=interval)

        if SAVEVIDEO:
            anim.save(f'{name}.mp4')
            return HTML(tag)
        else:
            return HTML(anim.to_jshtml())

    if not ANIM and not MKVIDEO:
        return HTML(tag)

### Ejemplos

In [ ]:
n = 1000
x = np.linspace(-4,5,n)
dx = x[1]-x[0]
Lap = mkLaplacian(n,dx)

In [ ]:
fun = wave(0*2*np.pi,x)
fun = fun * np.exp(-0.5*((x-0)/0.2)**2)
packet = Nor(fun)
#print(dx*sum(abs(packet)**2))

In [ ]:
H = - (hbar**2/2/(1*m))*Lap
prop = expmi(-1/hbar*H*0.02)

In [ ]:
ANIM = False
MKVIDEO = False
SAVEVIDEO = True

In [ ]:
metadisplay('demo1', makeAnimation, (packet,prop) )

In [ ]:
H = - (hbar**2/2/(10*m))*Lap
prop = expmi(-1/hbar*H*0.02)

In [ ]:
metadisplay('demo2', makeAnimation, (packet,prop) )

In [ ]:
n = 1000
x = np.linspace(-4,5,n)
dx = x[1]-x[0]
Lap = mkLaplacian(n,dx)

In [ ]:
packet = sum([a*wave(k,x) for a,k in zip(g,ks)])
packet = Nor(packet)
#print(dx*sum(abs(packet)**2))

In [ ]:
H = - (hbar**2/2/m)*Lap
prop = expmi(-1/hbar*H*0.005)

In [ ]:
metadisplay('demo3', makeAnimation, (packet,prop) )

In [ ]:
V = (x>1.5)*(x<2)

%matplotlib inline
plt.figure(figsize=(10,4))
plt.plot(x,abs(packet)**2)
plt.plot(x,V,'gray');

In [ ]:
dx * (packet.conj() @ H @ packet).sum()

In [ ]:
H = np.diag(150*V) - (hbar**2/2/m)*Lap
prop = expmi(-1/hbar*H*0.01)

In [ ]:
H = np.diag(60*V) - (hbar**2/2/m)*Lap
prop = expmi(-1/hbar*H*0.01)

metadisplay('demo4', makeAnimation2, (packet,prop) )

In [ ]:
V = (x>2) + (x<-2)

H = np.diag(150*V) - (hbar**2/2/m)*Lap
prop = expmi(-1/hbar*H*0.01)

metadisplay('demo5', makeAnimation2, (packet,prop) )

In [ ]:
V = x**2

H = np.diag(10*V) - (hbar**2/2/m)*Lap
prop = expmi(-1/hbar*H*0.01)

metadisplay('demo6', makeAnimation2, (packet,prop) )

In [ ]:
%matplotlib inline
plt.figure(figsize=(8,3))
plt.plot(x,abs((prop[150])),x,abs(prop[700]));

In [ ]:
plt.imshow(abs(prop));

In [ ]:
V = np.minimum(1.5,(x-1)/2)*(x>1)

plt.figure(figsize=(8,3))

plt.plot(x,abs(packet)**2)
plt.plot(x,V,'gray');

In [ ]:
H = np.diag(100*V) - (hbar**2/2/m)*Lap
prop = expmi(-1/hbar*H*0.01)

metadisplay('demo7', makeAnimation2, (packet,prop) )

In [ ]:
V = np.exp(-0.5*((x-2)/0.3)**2)

%matplotlib inline
plt.figure(figsize=(8,3))

plt.plot(x,abs(packet)**2)
plt.plot(x,V,'gray');

In [ ]:
H = np.diag(40*V) - (hbar**2/2/m)*Lap
prop = expmi(-1/hbar*H*0.01)

metadisplay('demo8', makeAnimation2, (packet,prop) )

In [ ]:
fun = wave(10,x)
fun = fun * np.exp(-0.5*((x+1)/0.4)**2)
fun = Nor(fun)

V = np.exp(-0.5*((x-1)/0.3)**2)

H = np.diag(40*V) - (hbar**2/2/m)*Lap
prop = expmi(-1/hbar*H*0.005)

mask = (1-2*np.exp(-0.5*((x+4+0.4)/0.4)**2))*(1-2*np.exp(-0.5*((x-5-0.4)/0.4)**2))

In [ ]:
%matplotlib inline
plt.plot(x,mask);

In [ ]:
metadisplay('demo9', makeAnimation3, (packet,prop,mask) )

In [ ]:
%matplotlib inline
plt.figure(figsize=(8,3))
plt.plot(x,abs((prop[150])),x,abs(prop[700]));

In [ ]:
plt.imshow(np.abs(prop));

In [ ]:
H = np.diag(40*V) - (hbar**2/2/(10*m))*Lap
prop = expmi(-1/hbar*H*0.01)
plt.figure(figsize=(8,3))
plt.plot(x,abs((prop[150])),x,abs(prop[700]));

In [ ]:
plt.imshow(np.abs(prop));

In [ ]:
plt.plot(np.real(prop[150,150:200]))
plt.plot(np.real(prop[700,700:750]));

In [ ]:
V = np.exp(-0.5*((x-2)/0.3)**2)
H = np.diag(40*V) - (hbar**2/2/m)*Lap
prop = expmi(-1/hbar*H*0.01)
prop1 = expmi(-1/hbar*(- (hbar**2/2/m)*Lap)*0.01)
prop2 = expmi(-1/hbar*(  np.diag(40*V)    )*0.01)

In [ ]:
plt.plot(x,np.diag(np.real(prop2)),x,np.diag(np.imag(prop2)));

In [ ]:
plt.imshow(abs(prop2));

In [ ]:
abs(prop - (prop2+prop1-(prop2@prop1-prop1@prop2))).max()

Una forma de interferencia:

In [ ]:
fun0 = wave(10,x)
fun = fun0 * (1*np.exp(-0.5*((x+1)/0.2)**2) + 1*np.exp(-0.5*((x-0)/0.2)**2))
fun2 = Nor(fun)
fun = fun0 * (1*np.exp(-0.5*((x+1)/0.2)**2) + 0*np.exp(-0.5*((x-0)/0.2)**2))
funa = Nor(fun)
fun = fun0 * (0*np.exp(-0.5*((x+1)/0.2)**2) + 1*np.exp(-0.5*((x-0)/0.2)**2))
funb = Nor(fun)

V = np.exp(-0.5*((x-1)/0.3)**2)

H = np.diag(40*V) - (hbar**2/2/(m))*Lap
prop = expmi(-1/hbar*H*0.005)

mask = (1-2*np.exp(-0.5*((x+4+0.4)/0.4)**2))*(1-2*np.exp(-0.5*((x-5-0.4)/0.4)**2))
prop = np.diag(mask)@prop

In [ ]:
def makeAnimation4(nframes=100, init=None, video=True):

    fig, ax = plt.subplots(figsize=(8,4))
    if video: plt.close()

    ax.set_xlim(( -4, 5))
    ax.set_ylim((-0.05, 2))

    line1, = ax.plot([], [], color='green', alpha=0.5)
    line2, = ax.plot([], [], color='blue', alpha=0.5)
    line4, = ax.plot([], [], color='red')
    line3, = ax.plot([], [], '-',lw=2)

    ax.plot(x, V, 'gray')

    ax.plot(x,mask,'pink')

    r  = fun2
    r1 = funa
    r2 = funb

    def fotogram(i):
        nonlocal r, r1, r2
        line3.set_data(x,abs(r)**2)
        line1.set_data(x,abs(r1)**2)
        line2.set_data(x,abs(r2)**2)
        line4.set_data(x,0.5*(abs(r1)**2 + abs(r2)**2))
        if i > 0:
            r  = prop@r
            r1 = prop@r1
            r2 = prop@r2
        return ()

    if init is not None:
        fotogram(init)

    return metaAnimation(fig,fotogram,nframes,video)

In [ ]:
%matplotlib widget
metadisplay('demo10', makeAnimation4, () )

## Comentarios

Parece que se pueden encontrar las energías discretas de un estado ligado expresando en forma matricial el hamiltoniano y calculando directamente sus valores y vectores propios. Los valores no son exactamente los mismos que los teóricos. La discrepancia puede deberse a la imperfecta discretización del operador laplaciano, o a las condiciones de frontera. Pero en cualquier caso, luego debería haber un continuo posible de energías, que no sé a que eigenvectors deberían corresponder.

En la evolución dinámica una discretización ingenua (Euler) no hace evolucionar un paquete de ondas correctamente. Pero sin embargo, con la matriz exponencial sí que se consigue un operador unitario que hace avanzar la función de onda. Hay que elegir el intervalo temporal. Supongo que debe ser pequeño, pero la exponencial en principio podría con todo...

Esto sí funciona, y parece comportarse bien con barreras de potencial y cosas así. El problema es que las condiciones de contorno tengo que ponerlas periódicas porque si no rebota como si fuera un dominio finito. Si la transformación es unitaria no veo cómo se podría simular el efecto de que la partícula se sale del dominio. He hecho algunas pruebas con una función de ventana para anular en los extremos, no sé si es correcto.

Por otra parte la exponencial es (ingenuamente) muy costosa de calcular si quiero una discretización fina. No si la implementamos con una factorización, aprovechando que, aunque $i/\hbar H t$ no es hermítica, la parte $1/\hbar H t$ sí lo es, y podemos usar Euler (!?)

Parece que tienen sentido los resultados. El oscilador armónico es realmente curioso. Aparentemente no tiene dispersión, lo que me resulta contraintuitivo, pero parece corroborarlo [esta animación](https://www.st-andrews.ac.uk/physics/quvis/simulations_phys/ph22_Oscillator_Wave_Packet.swf) y estas [transparencias](http://www.chemie.unibas.ch/~tulej/Spectroscopy_related_aspects/Lecture18_Spec_Rel_Asp.pdf) que mencionan que esto ya lo propuso Schrödinger. Está documentado en muchos más sitios.

He comprobado que la matriz exponencial cumple perfectamente la condición de composición, de modo que la evolución unitaria puede conseguirse tranquilamente para intervalos de tiempo más grandes. (No sé hasta cuánto aguantará realmente, sería interesante verlo. Teniendo en cuenta que al final se reduce a senos y cosenos tal vez no tenga límite.)

Y me surge la pregunta: ¿puede hacerse lo mismo en cualquier otra ecuación diferencial? Si es lineal, sí: Si $\dot{ \vec y}(t) = A \vec y(t)$ entonces $\vec y(t)=e^{At}\vec y(0)$. (También se pueden resolver sistemas no homogéneos de forma parecida).

Otra observación interesante es que el operador de evolución tiene una estructura no densa, sino en forma de banda. Esto significa que a cada posición le alcanza la influencia de puntos vecinos hasta una cierta distancia. Esta distancia depende del intervalo de evolución que se ha integrado en la exponencial, y también, según he comprobado, de la masa. La velocidad de difusión de la incertidumbre aumenta la reducirse la masa de la partícula ([wave packet spreading](https://en.wikipedia.org/wiki/Wave_packet#Free_propagator)). O sea, hay un kernel, y se produce algo parecido a una convolución (aunque no constante).

Entonces, tal vez podemos desdoblar el propagador y crear una simulación de dominio no acotado, donde podamos dejar salir la onda por los extremos. Aunque en el fondo creo que no queda otra solución que atenuar la función suavemente (para que no "rebote" con oscilaciones) en los extremos. Posiblemente sea aceptable.

Otra idea era descomponer el propagador en la zona libre y la de potencial, pero no commutan y entonces se complica más que recalcular.